# GRT Parallel Processing

In this notebook we will test the parallel processing for GRT

The following two lines are very important for getting the code working as expected in a multicore machine.  Since numpy uses OpenMP to compute matricial operations, the program normally spawns child processes to perform those operations.  As a result if you use multiprocessing (mp), besides the processes retrieved by mp, it will start 4 NP processes.  To avoid this, you just need to set this variable.

In [8]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

In [13]:
from gravray import *
from gravray.util import *
from gravray.sampling import *
from gravray.spice import *
from gravray.orbit import *
from gravray.stats import *

import gc
from tqdm import tqdm
import pandas as pd
import multiprocessing as mp
from itertools import product as cartesian

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Initialize

In [25]:
Spice.loadKernels()
NP=mp.cpu_count()-1
print("Available number of processors: ",NP)

Available number of processors:  3


## Common data

In [26]:
body="EARTH"
earth=Body(body)

## Parallel function

In [27]:
NRAYS=0
def rayProcessing(initial,nw=0):
    global NRAYS
    t=initial[0]
    site=initial[1]
    direction=initial[2]
    ray=GrtRay(site,direction[0][0],direction[0][1],direction[1])
    ray.updateRay(t)
    try:
        ray.propagateRay(t)
        #detJ=ray.calcJacobianDeterminant()
        detJ=1
    except AssertionError as e:
        detJ=0
    raydf=ray.packRay()
    raydf["detJ"]=detJ
    del ray
    NRAYS+=1
    if (NRAYS%10000)==1:
        unreachable=gc.collect()
        print(f"Completed rays by worker {nw}: {NRAYS} (unreachable {unreachable})")
        elTime(start=True)
    return raydf
    
def rayProcessingMulti(initials,nw=0):
    print(f"Processing {len(initials)} initial conditions in worker {nw}")
    raydfs=[rayProcessing(initial) for initial in initials]
    return raydfs

allrays=pd.DataFrame()
def joinResults(raydfs):
    global allrays
    allrays=pd.concat((allrays,)+tuple(raydfs))
    del raydfs
    gc.collect()

## Test data

In [28]:
ts=[Spice.str2t("02/15/2013 03:20:34 UTC")]
siteprops=[[61.1*Angle.Deg,54.8*Angle.Deg,23.3*Const.km]]
sites=[]
for siteprop in siteprops:
    sites+=[Location(earth,siteprop[0],siteprop[1],siteprop[2])]
directions=[[[101.1*Angle.Deg,15.9*Angle.Deg],-18.6*Const.km/Const.s]]

#List of conditions
initials=list(cartesian(*[ts,sites,directions]))
rayProcessingMulti(initials)

Processing 1 initial conditions in worker 0
Completed rays by worker 0: 1 (unreachable 7)
Elapsed time since script start: 3.94697 min


[            et   lon   lat      alt      A     h        v          ximp  \
 0  414170434.0  61.1  54.8  23300.0  101.1  15.9 -18600.0 -1.232675e+11   
 
            yimp          zimp  ...       vzhel         q         e         i  \
 0  8.134799e+10 -3.528140e+06  ...  -2274.5822  0.738582  0.549668  4.041579   
 
             W          w          M         a             n  detJ  
 0  326.572556  106.86342  21.323547  1.640082  9.479146e-08     1  
 
 [1 rows x 28 columns]]

## Massive input data

In [29]:
#Numbers
Ntimes=Nsites=Npoints=Nvels=10
"""
Ntimes=365
Nsites=100
Npoints=100
Nvels=50
#"""

#Times
print("Preparing times...")
tini=Spice.str2t("02/15/2013 03:20:34")
tend=tini+Const.Year
ts=np.linspace(tini,tend,10)

#Sites
print("Preparing sites...")
elTime(0)
H=23.3*Const.km
points=Sample(Nsites)
points.genUnitSphere()
siteprops=np.zeros((Nsites,3))
siteprops[:,:2]=points.pp[:,1:]
siteprops[:,2]=H*np.ones(Nsites)
sites=[]
for siteprop in siteprops:
    sites+=[Location(earth,siteprop[0],siteprop[1],siteprop[2])]
elTime()

#Directions
print("Preparing directions...")
elTime(0)
gpoints=Sample(Npoints)
gpoints.genUnitHemisphere()
speeds=-np.linspace(11.2,72.0,Nvels)*Const.km/Const.s
directions=list(cartesian(*[gpoints.pp[:,1:].tolist(),speeds]))
elTime()

#Initial conditions
print("Preparing initial conditions...")
elTime(0)
initials=list(cartesian(*[ts,sites,directions]))
elTime()

Ninitials=len(initials)
print(f"Number of initial conditions: {Ninitials} = {len(ts)}(ts)*{len(sites)}(sites)*{len(directions)}(dirs)")

Preparing times...
Preparing sites...
Elapsed time since last call: 1.80292 ms
Preparing directions...
Elapsed time since last call: 628.233 us
Preparing initial conditions...
Elapsed time since last call: 88.6917 us
Number of initial conditions: 80 = 10(ts)*2(sites)*4(dirs)


## Chunking and computing time estimations

In [30]:
#Computing time estimations
print(f"Sequential processing of {Ninitials} rays:")
dt,dtu=elTime(0)
tinitials=initials[:10]
rays=rayProcessingMulti(tinitials)
dt,dtu=elTime()
tpray=dt/len(tinitials)
tupray=tUnit(tpray)
totrays=tpray*Ninitials
toturays=tUnit(totrays)
print(f"Total duration: {dtu[0]} {dtu[1]}, Duration per ray: {tupray[0]} {tupray[1]}")

#Chunks
npchunk=np.int(np.ceil(Ninitials/NP))
cinitials=[initial for initial in Util.chunkList(initials,npchunk)]
Nchunks=len(cinitials)
print(f"{Nchunks} chunks containing {npchunk} initial conditions")
tchunk=tpray*npchunk
tchunku=tUnit(tchunk)
print()
print(f"Estimated total: {toturays[0]} {toturays[1]}")
print(f"Estimated time per chunk (estimated parallel): {tchunku[0]} {tchunku[1]}")

Sequential processing of 80 rays:
Processing 10 initial conditions in worker 0
Elapsed time since last call: 57.2081 ms
Total duration: 57.20806121826172 ms, Duration per ray: 5.720806121826172 ms
3 chunks containing 27 initial conditions

Estimated total: 457.66448974609375 ms
Estimated time per chunk (estimated parallel): 154.46176528930664 ms


## Parallel processing

In [31]:
allrays=pd.DataFrame()
pool=mp.Pool(processes=NP)
elTime(0)
[pool.apply_async(rayProcessingMulti,args=(inis,nw),callback=joinResults) for nw,inis in enumerate(cinitials)]
pool.close()
pool.join()
elTime()

Processing 27 initial conditions in worker 0
Processing 27 initial conditions in worker 1
Processing 26 initial conditions in worker 2
Elapsed time since last call: 445.189 ms


(0.4451887607574463, [445.1887607574463, 'ms'])

In [32]:
print("Number of results:",len(allrays))

Number of results: 80


In [ ]:
allrays.to_csv("rays_parallel.csv")